# 🎤 Fine-Tuning StyleTTS2 pour le Darija

## Version Optimisée GitHub + HuggingFace (Setup rapide ~3 min)

**Sources:**
- 📦 **Code:** `github.com/VOTRE-USERNAME/arable-tts` ⬅️ Remplacez par votre username GitHub
- 🎵 **Audio:** `huggingface.co/datasets/VOTRE-USERNAME/darija-dataset` ⬅️ Remplacez par votre repo HF

### Prérequis:
- GPU: T4, V100, ou A100 (minimum 16GB VRAM)
- Dataset déjà uploadé sur Hugging Face

## 📝 Configuration - MODIFIEZ ICI

In [ ]:
# 🔧 CONFIGURATION - Modifiez ces valeurs
GITHUB_REPO = "VOTRE-USERNAME/arable-tts"  # Votre repo GitHub
HF_DATASET_REPO = "VOTRE-USERNAME/darija-dataset"  # Votre dataset HuggingFace
HF_TOKEN = ""  # Optionnel, seulement si dataset privé

print(f"✅ GitHub: {GITHUB_REPO}")
print(f"✅ HuggingFace: {HF_DATASET_REPO}")

## 1. Vérification GPU

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Installation des dépendances

In [ ]:
!pip install -q phonemizer==3.2.1 munch accelerate pydub nltk g2p_en num2words inflect unidecode pyyaml librosa scipy matplotlib soundfile
!pip install -q torch torchaudio transformers einops einops-exts tqdm omegaconf huggingface_hub
!pip install -q git+https://github.com/resemble-ai/monotonic_align.git
!apt-get install -qq espeak-ng
print("✅ Dépendances installées!")

## 3. Téléchargement du code (GitHub) et audio (HuggingFace)

In [ ]:
import os
from huggingface_hub import snapshot_download

# Cloner StyleTTS2
if not os.path.exists("/content/StyleTTS2"):
    !git clone https://github.com/yl4579/StyleTTS2.git /content/StyleTTS2
    print("✅ StyleTTS2 cloné!")
else:
    print("ℹ️ StyleTTS2 déjà présent")

# Cloner votre repo avec les configs
if not os.path.exists("/content/arable-tts"):
    !git clone https://github.com/{GITHUB_REPO}.git /content/arable-tts
    print("✅ Configs clonées depuis GitHub!")
else:
    print("ℹ️ Repo déjà présent")

# Télécharger les audio depuis HuggingFace
print(f"📥 Téléchargement audio depuis HuggingFace: {HF_DATASET_REPO}...")
snapshot_download(
    repo_id=HF_DATASET_REPO,
    repo_type="dataset",
    local_dir="/content/dataset_darija",
    local_dir_use_symlinks=False,
    token=HF_TOKEN if HF_TOKEN else None
)
print("✅ Dataset audio téléchargé!")

# Vérifier ce qui a été téléchargé
if os.path.exists("/content/dataset_darija/wavs"):
    wav_count = len([f for f in os.listdir("/content/dataset_darija/wavs") if f.endswith('.wav')])
    print(f"🎵 {wav_count} fichiers audio détectés")

metadata_files = ["metadata_train.csv", "metadata_eval.csv", "metadata.json"]
for mf in metadata_files:
    if os.path.exists(f"/content/dataset_darija/{mf}"):
        print(f"📄 {mf} présent")

## 4. Téléchargement des modèles pré-entraînés

In [ ]:
!mkdir -p /content/StyleTTS2/Models/LibriTTS /content/StyleTTS2/Models/Darija
!mkdir -p /content/StyleTTS2/Utils/JDC /content/StyleTTS2/Utils/ASR /content/StyleTTS2/Utils/PLBERT

print("📥 Téléchargement des modèles pré-entraînés...")

# Modèle principal
!wget -q --show-progress -O /content/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Models/LibriTTS/epochs_2nd_00020.pth
!wget -q -O /content/StyleTTS2/Models/LibriTTS/config.yml \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Models/LibriTTS/config.yml

# JDC (pitch extractor)
!wget -q --show-progress -O /content/StyleTTS2/Utils/JDC/bst.t7 \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Utils/JDC/bst.t7

# ASR (speech recognition)
!wget -q --show-progress -O /content/StyleTTS2/Utils/ASR/epoch_00080.pth \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Utils/ASR/epoch_00080.pth
!wget -q -O /content/StyleTTS2/Utils/ASR/config.yml \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Utils/ASR/config.yml

# PLBERT (phoneme encoder)
!wget -q --show-progress -O /content/StyleTTS2/Utils/PLBERT/step_1000000.t7 \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Utils/PLBERT/step_1000000.t7
!wget -q -O /content/StyleTTS2/Utils/PLBERT/config.yml \
    https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Utils/PLBERT/config.yml

print("✅ Modèles pré-entraînés téléchargés!")

## 5. Patches PyTorch 2.6+ (CRITIQUE!)

In [ ]:
import re, os

files_to_patch = [
    "/content/StyleTTS2/models.py",
    "/content/StyleTTS2/Utils/ASR/models.py",
    "/content/StyleTTS2/Utils/JDC/model.py",
    "/content/StyleTTS2/Utils/PLBERT/util.py",
    "/content/StyleTTS2/meldataset.py",
]

for filepath in files_to_patch:
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            content = f.read()
        if 'torch.load' in content and 'weights_only' not in content:
            new_content = re.sub(r'torch\.load\(([^)]+)\)', r'torch.load(\1, weights_only=False)', content)
            with open(filepath, 'w') as f:
                f.write(new_content)
            print(f"✅ Patched: {os.path.basename(filepath)}")
        else:
            print(f"ℹ️ OK: {os.path.basename(filepath)}")

print("\n✅ Patches PyTorch appliqués!")

## 6. Préparation des données

In [ ]:
import json, random, os, shutil

os.makedirs("/content/StyleTTS2/Data", exist_ok=True)
os.makedirs("/content/StyleTTS2/wavs", exist_ok=True)

# Copier les fichiers audio
print("🎵 Copie des fichiers audio...")
source_wavs = "/content/dataset_darija/wavs"
dest_wavs = "/content/StyleTTS2/wavs"

if os.path.exists(source_wavs):
    for f in os.listdir(source_wavs):
        if f.endswith('.wav'):
            shutil.copy(f"{source_wavs}/{f}", f"{dest_wavs}/{f}")
    print(f"✅ {len(os.listdir(dest_wavs))} fichiers audio copiés")
else:
    print("❌ Dossier wavs introuvable!")

# Créer train/val lists depuis metadata.json
metadata_path = "/content/dataset_darija/metadata.json"
if os.path.exists(metadata_path):
    with open(metadata_path, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    random.seed(42)
    random.shuffle(metadata)
    split = int(len(metadata) * 0.95)
    train_data, eval_data = metadata[:split], metadata[split:]

    with open('/content/StyleTTS2/Data/train_list.txt', 'w', encoding='utf-8') as f:
        for item in train_data:
            f.write(f"{item['audio_file']}|{item['text'].replace(chr(10), ' ')}|0\n")

    with open('/content/StyleTTS2/Data/val_list.txt', 'w', encoding='utf-8') as f:
        for item in eval_data:
            f.write(f"{item['audio_file']}|{item['text'].replace(chr(10), ' ')}|0\n")

    print(f"✅ Train: {len(train_data)} samples")
    print(f"✅ Val: {len(eval_data)} samples")
else:
    print("❌ metadata.json introuvable!")

## 7. Configuration du training

In [ ]:
import yaml, os

config = {
    'log_dir': 'Models/Darija',
    'save_freq': 10,
    'device': 'cuda',
    'epochs_1st': 0,
    'epochs_2nd': 80,
    'batch_size': 4,
    'max_len': 400,
    'pretrained_model': 'Models/LibriTTS/epochs_2nd_00020.pth',
    'data_params': {
        'train_data': 'Data/train_list.txt',
        'val_data': 'Data/val_list.txt',
        'root_path': '',
        'OOD_data': 'Data/OOD_texts.txt',
        'min_length': 50,
        'sample_rate': 24000,
    },
    'preprocess_params': {
        'sr': 24000,
        'spect_params': {
            'n_fft': 2048,
            'win_length': 1200,
            'hop_length': 300,
            'n_mels': 80
        }
    },
    'model_params': {
        'multispeaker': False,
        'dim_in': 64,
        'hidden_dim': 512,
        'max_conv_dim': 512,
        'n_layer': 3,
        'n_mels': 80,
        'n_token': 178,
        'max_dur': 50,
        'style_dim': 128,
        'dropout': 0.2,
        'decoder': {
            'type': 'istftnet',
            'resblock_kernel_sizes': [3, 7, 11],
            'upsample_rates': [10, 6],
            'upsample_initial_channel': 512,
            'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
            'upsample_kernel_sizes': [20, 12],
            'gen_istft_n_fft': 20,
            'gen_istft_hop_size': 5
        },
        'slm': {
            'model': 'microsoft/wavlm-base-plus',
            'sr': 16000,
            'hidden': 768,
            'nlayers': 13,
            'initial_channel': 64
        },
        'diffusion': {
            'embedding_mask_proba': 0.1,
            'transformer': {
                'num_layers': 3,
                'num_heads': 8,
                'head_features': 64,
                'multiplier': 2
            },
            'dist': {
                'sigma_data': 0.2,
                'estimate_sigma_data': True,
                'mean': -3.0,
                'std': 1.0
            }
        }
    },
    'loss_params': {
        'lambda_mel': 5.0,
        'lambda_gen': 1.0,
        'lambda_slm': 1.0,
        'lambda_mono': 1.0,
        'lambda_s2s': 1.0,
        'lambda_F0': 1.0,
        'lambda_norm': 1.0,
        'lambda_dur': 1.0,
        'lambda_ce': 20.0,
        'lambda_sty': 1.0,
        'lambda_diff': 1.0,
        'diff_epoch': 20,
        'joint_epoch': 40
    },
    'optimizer_params': {'lr': 0.0001},
    'slmadv_params': {
        'min_len': 400,
        'max_len': 500,
        'batch_percentage': 0.5,
        'iter': 10,
        'thresh': 5.0,
        'scale': 0.01,
        'sig': 1.5
    },
    'F0_path': 'Utils/JDC/bst.t7',
    'ASR_config': 'Utils/ASR/config.yml',
    'ASR_path': 'Utils/ASR/epoch_00080.pth',
    'PLBERT_dir': 'Utils/PLBERT/',
}

os.makedirs('/content/StyleTTS2/Configs', exist_ok=True)
with open('/content/StyleTTS2/Configs/config_darija_ft.yml', 'w') as f:
    yaml.dump(config, f)

print("✅ Configuration créée!")

## 8. Vérification finale avant training

In [ ]:
import os

checks = {
    "Config": "/content/StyleTTS2/Configs/config_darija_ft.yml",
    "Audio (wavs)": "/content/StyleTTS2/wavs",
    "Train list": "/content/StyleTTS2/Data/train_list.txt",
    "Val list": "/content/StyleTTS2/Data/val_list.txt",
    "Modèle LibriTTS": "/content/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth",
    "JDC": "/content/StyleTTS2/Utils/JDC/bst.t7",
    "ASR": "/content/StyleTTS2/Utils/ASR/epoch_00080.pth",
    "PLBERT": "/content/StyleTTS2/Utils/PLBERT/step_1000000.t7",
}

all_ok = True
for name, path in checks.items():
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"{status} {name}")
    if not exists:
        all_ok = False

# Compter les fichiers
if os.path.exists("/content/StyleTTS2/wavs"):
    wav_count = len([f for f in os.listdir("/content/StyleTTS2/wavs") if f.endswith('.wav')])
    print(f"\n🎵 {wav_count} fichiers audio prêts")

if all_ok:
    print("\n🎉 TOUT EST PRÊT! Vous pouvez lancer le training!")
else:
    print("\n⚠️ Certains fichiers manquent. Vérifiez les étapes précédentes.")

## 9. 🚀 Lancement du Fine-Tuning

⚠️ **Durée estimée: 8-12h sur GPU T4**

Checkpoints sauvegardés tous les 10 epochs dans `/content/StyleTTS2/Models/Darija/`

In [ ]:
%cd /content/StyleTTS2
!accelerate launch --mixed_precision=no --num_processes=1 train_finetune.py --config_path ./Configs/config_darija_ft.yml

## 10. Sauvegarde sur Google Drive (IMPORTANT!)

In [ ]:
from google.colab import drive
import shutil
import os

# Monter Google Drive
drive.mount('/content/drive')

# Créer le dossier de sauvegarde
backup_dir = '/content/drive/MyDrive/darija_checkpoints'
os.makedirs(backup_dir, exist_ok=True)

# Copier les checkpoints
print("💾 Sauvegarde des checkpoints sur Google Drive...")
source_dir = '/content/StyleTTS2/Models/Darija'
if os.path.exists(source_dir):
    for item in os.listdir(source_dir):
        source_path = os.path.join(source_dir, item)
        dest_path = os.path.join(backup_dir, item)
        if os.path.isfile(source_path):
            shutil.copy2(source_path, dest_path)
            print(f"✅ {item}")
    print("\n🎉 Checkpoints sauvegardés sur Drive!")
else:
    print("⚠️ Aucun checkpoint trouvé")

## 11. Upload du modèle entraîné sur Hugging Face (optionnel)

In [ ]:
from huggingface_hub import HfApi, login

# Configuration
HF_MODEL_REPO = "VOTRE-USERNAME/darija-styletts2"  # Changez ici
HF_TOKEN_UPLOAD = ""  # Votre token HF avec droits write

if HF_TOKEN_UPLOAD:
    print("🔐 Connexion à Hugging Face...")
    login(token=HF_TOKEN_UPLOAD)
    
    api = HfApi()
    
    # Créer le repo
    api.create_repo(repo_id=HF_MODEL_REPO, repo_type="model", exist_ok=True)
    
    # Upload les checkpoints
    print(f"📤 Upload vers {HF_MODEL_REPO}...")
    api.upload_folder(
        folder_path="/content/StyleTTS2/Models/Darija",
        repo_id=HF_MODEL_REPO,
        repo_type="model"
    )
    
    print(f"✅ Modèle disponible sur: https://huggingface.co/{HF_MODEL_REPO}")
else:
    print("ℹ️ Token HF non fourni, upload ignoré")